<div align="center">
  <h1>Dashboard</h1>
  <h2>Data Hunian Rumah dan Apartemen di Jakarta Pusat</h2>
</div>

In [1]:
## jika packages berikut belum terinstall, un-comment dan jalankan cell ini untuk menginstallnya
# %pip install pandas numpy plotly

# note: jika ada yang belum terinstall lakukan command di cell baru:
# %pip install <nama package>

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display, Markdown

<div align="center">
  <h3>OVERVIEW</h3>
</div>

- Data hunian

In [3]:
hunian = pd.read_json('../Data Scraping/data/hunian.json')
hunian.head(5)

id_iklan tipe_properti  luas_bangunan  kamar_tidur  kamar_mandi  \
0   aps3187118     Apartemen            112            3            2   
1   aps3187337     Apartemen             33            2            1   
2   aps3187554     Apartemen             50            2            1   
3   aps3187588     Apartemen             79            1            1   
4  hos16255795         Rumah             80            3            3   

          lokasi                     sertifikat tipe_iklan  \
0        Menteng     SHM - Sertifikat Hak Milik       jual   
1  Cempaka Putih  Lainnya (PPJB,Girik,Adat,dll)       jual   
2    Sawah Besar  Lainnya (PPJB,Girik,Adat,dll)       jual   
3      Kemayoran     SHM - Sertifikat Hak Milik       jual   
4  Gunung Sahari     SHM - Sertifikat Hak Milik       jual   

  periode_kepemilikan       harga  diperbarui  id_agen  
0             Pemilik  5500000000  2024-05-27  1859657  
1             Pemilik   550000000  2024-05-09   947615  
2             Pemilik   800000000  2024-02-02  1527108  
3             Pemilik  2400000000  2024-02-02    48760  
4             Pemilik  1000000000  2024-02-02  1356696

- Data rumah

In [4]:
rumah = pd.read_json('../Data Scraping/data/rumah.json')
rumah.head(5)

id_iklan  luas_tanah  carport  taman
0  hos16255795          49        1  Tidak
1  hos16255835         400        2  Tidak
2  hos16251736         378        0     Ya
3  hos16252133         376        1  Tidak
4  hos16252488         515        0  Tidak

- Data apartemen

In [5]:
apartemen = pd.read_json('../Data Scraping/data/apartemen.json')
apartemen.head(5)

id_iklan kondisi_properti kondisi_perabotan
0  aps3187118            Bagus         Furnished
1  aps3187337            Bagus         Furnished
2  aps3187554            Bagus         Furnished
3  aps3187588            Bagus         Furnished
4  aps3187135            Bagus         Furnished

- Data agen

In [6]:
agen = pd.read_json('../Data Scraping/data/agen.json')
agen.head(5)

id_agen                  nama_agen  nomor_telepon  terjual  tersewa  \
0  1859657                     Grace   6287736666636        0        0   
1   947615           Junison Siagian   6282368494901       41       61   
2  1527108                  maria lie    62811135705        3        3   
3    48760                Vemby Intan   628996060307        2        0   
4  1356696  novie asiaonehi asiaonehi  6287810000275        1        1   

          nama_perusahaan  
0    Xavier Marks Menteng  
1                  WM PRO  
2  Asia One Harapan Indah  
3              Dream Home  
4  Asia One Harapan Indah

- Data perusahaan

In [7]:
perusahaan = pd.read_json('../Data Scraping/data/perusahaan.json')
perusahaan.head(5)

nama_perusahaan                                             alamat
0    Xavier Marks Menteng  Jl. Cikini Raya No.121C, RW.4, Cikini, Kec. Me...
1                  WM PRO  Apartemen Taman Rasuna Kawasan Epicentrum,Kuni...
2  Asia One Harapan Indah  Rukan Mega Bulevard Blok RV-2 No. 6A, Medan Sa...
3              Dream Home  Ruko Alexandrite Blok ALX 3 No. 17 Gading Serp...
4            ThayProperty     Jl. Pondok Kelapa, Duren Sawit, Jakarta Timur

<div align="center">
  <h3>VISUALIZATION</h3>
</div>

In [8]:
def calculate_harga_per_bulan(row):
    if row['periode_kepemilikan'] == 'Harian':
        return row['harga'] * 30
    elif row['periode_kepemilikan'] == 'Tahunan':
        return row['harga'] / 12
    else:
        return row['harga']
    
def bar(df, column1, column2, x_, title):
    dist = df.groupby([column1, column2]).size().reset_index(name='count')
    fig = px.bar(dist, x=column1, y='count', color=column2, title=title + ' - snip untuk zoom agar lebih jelas',
                labels={column1: x_, 'count': 'Count', column2:'Property Type'},
                color_discrete_sequence=px.colors.qualitative.Pastel1)
    fig.show()

def pie(df, column, donut, title):
    dist = df[column].value_counts().reset_index()
    dist.columns = [column, 'count']

    fig = px.pie(dist, names=column, values='count', title=title,
                color_discrete_sequence=px.colors.sequential.Redor)
    fig.update_traces(textinfo='label+percent', textposition='outside',
                      pull=[0.1] * len(dist), hole=0.5)
    fig.update_layout(showlegend=False, annotations=[dict(text=donut, font_size=20, showarrow=False)])
    fig.show()

def scatter(df, title, x_, color_):
    fig = px.scatter(df, x=x_, y='harga_per_bulan', color=color_, 
                    title=title + ' - snip untuk zoom agar lebih jelas', 
                    labels={'luas_bangunan': 'Building Area (m\u00b2)', 'harga_per_bulan': 'Price', 'tipe_properti': 'Property Type'})
    fig.show()

def boxplot(df, column, title):
    fig = px.box(df, y=column, title=title + ' - snip untuk zoom agar lebih jelas',)
    fig.update_layout(yaxis_title='Area (m²)')
    fig.show()

In [9]:
df = pd.read_json('../Data Scraping/data/data_clean.json') 

# HUNIAN
display(Markdown("<div align='center'><h2>====================</h2></div>"))
display(Markdown("<div align='center'><h2>Informasi Hunian</h2></div>"))

title = 'Boxplot Luas Bangunan Hunian'
boxplot(df, 'luas_bangunan', title)

title = 'Distribusi Jumlah Kamar Tidur di Hunian'
bar(df, 'kamar_tidur', 'tipe_properti', 'Bedrooms', title)

title = 'Distribusi Jumlah Kamar Mandi di Hunian'
bar(df, 'kamar_mandi', 'tipe_properti', 'Bathrooms', title)

title = 'Distribusi Sertifikat Hunian'
bar(df, 'sertifikat', 'tipe_properti', 'Certificate', title)

# RUMAH
display(Markdown("<div align='center'><h2>====================</h2></div>"))
display(Markdown("<div align='center'><h2>Informasi Rumah</h2></div>"))
rumah = df[df['tipe_properti'] == 'Rumah']

title = 'Boxplot Luas Tanah Rumah'
boxplot(df, 'luas_tanah', title)

title = 'Distribusi Ketersediaan Taman di Rumah'
pie(rumah, 'taman', 'Garden', title)

title = 'Distribusi Jumlah Carport di Rumah'
dist = rumah.groupby('carport').size().reset_index(name='count')
fig = px.bar(dist, x='carport', y='count', title=title + ' - snip untuk zoom agar lebih jelas',
            labels={'count': 'Count'},
            color_discrete_sequence=px.colors.qualitative.Pastel1)
fig.show()

# APARTEMEN
display(Markdown("<div align='center'><h2>====================</h2></div>"))
display(Markdown("<div align='center'><h2>Informasi Apartemen</h2></div>"))
apartemen = df[df['tipe_properti'] == 'Apartemen']

title = 'Distribusi Kondisi Properti'
pie(apartemen, 'kondisi_properti', 'Condition', title)

title = 'Distribusi Kondisi Perabotan'
pie(apartemen, 'kondisi_perabotan', 'Furniture', title)

# AGEN
display(Markdown("<div align='center'><h2>====================</h2></div>"))
display(Markdown("<div align='center'><h2>Informasi Agen</h2></div>"))
df['affiliate'] = np.where(df['nama_perusahaan'].notnull(), 'affiliated', 'independent')
title = 'Distribusi Status Agen'
pie(df, 'affiliate', 'Affiliation', title)

# TOP 5 AGEN
display(Markdown("<div align='center'><h2>====================</h2></div>"))
agen_summary = pd.read_json('../Data Scraping/data/agen.json')
agen_summary['total'] = agen_summary['terjual'] + agen_summary['tersewa']
top_5_agen = agen_summary.sort_values(by='total', ascending=False).head(5)
styled_df = top_5_agen[['nama_agen', 'nama_perusahaan', 'terjual', 'tersewa']].style.hide(axis='index')
display(Markdown("<div align='center'><h3>Top 5 Agents by Total Sales and Rentals</h3></div>"))
display(styled_df)

# DISEWAKAN
display(Markdown("<div align='center'><h2>====================</h2></div>"))
display(Markdown("<div align='center'><h2>Informasi Properti Disewakan</h2></div>"))
df['harga_per_bulan'] = df.apply(calculate_harga_per_bulan, axis=1)
sewa_df = df[df['tipe_iklan'] == 'sewa']
title = 'Distribusi Periode Sewa'
pie(sewa_df, 'periode_kepemilikan', 'period', title)
display(Markdown("**Harga per Bulan vs Luas Bangunan**"))
scatter(sewa_df, title='Price per Month vs Building Area by Property Type (For Rent)', x_='luas_bangunan', color_='tipe_properti')
scatter(sewa_df, title='Price per Month vs Certificate (For Rent)', x_='sertifikat', color_='sertifikat')

# DIJUAL
display(Markdown("<div align='center'><h2>====================</h2></div>"))
display(Markdown("<div align='center'><h2>Informasi Properti Dijual</h2></div>"))
jual_df = df[df['tipe_iklan'] == 'jual']
display(Markdown("**Harga vs Luas Bangunan**"))
scatter(jual_df, title='Price vs Building Area by Property Type (For Sale)', x_='luas_bangunan', color_='tipe_properti')
scatter(jual_df, title='Price vs Certificate (For Sale)', x_='sertifikat', color_='sertifikat')

<div align='center'><h2>====================</h2></div>

<div align='center'><h2>Informasi Hunian</h2></div>

<div align='center'><h2>====================</h2></div>

<div align='center'><h2>Informasi Rumah</h2></div>

<div align='center'><h2>====================</h2></div>

...

<div align='center'><h2>Informasi Properti Disewakan</h2></div>

**Harga per Bulan vs Luas Bangunan**

<div align='center'><h2>====================</h2></div>

<div align='center'><h2>Informasi Properti Dijual</h2></div>

**Harga vs Luas Bangunan**